In [2]:
import pandas as pd


In [2]:
article = pd.read_csv("Articles.csv")
article_name = "Desktop Computer"

In [16]:
article[article["Articles"]==article_name]["Cost per unit"].tolist()[0]

20500.0

In [11]:
public1 = pd.read_excel("DISTRICT AND PUBLIC DATA BASE 31012024.xlsx",sheet_name="DO NOT TOUCH_2021 to 23 data")
public1



,AADHAR No,AADHAR No.1,Len,NAME,BENEFICIARY ITEM,YEAR,Category,Selection Status\nYes / No
0,9292 9645 9071,929296459071,12.0,M.KARTHIKEYAN,Push cart with Top,2021,NaN,NaN
1,2062 6807 0869,206268070869,12.0,S.RAJESHWARI,Battery and Charger,2021,NaN,NaN
2,2593 5886 7697,259358867697,12.0,M.PAECHIMUTHU,Powered Sewing Machine,2021,NaN,NaN
3,3636 6154 9968,363661549968,12.0,M.DEVI,Sewing Machine,2021,NaN,NaN
4,5263 3195 0205,526331950205,12.0,A.THIYAGARAJAN,Push cart with Top,2021,NaN,NaN
...,...,...,...,...,...,...,...,...
1008,NaN,450026695024,NaN,S.Hari,Gents Cycle,2023,Public,Yes
1009,NaN,331749803100,NaN,M.Bagyalakshmi,Girls Cycle,2023,Public,Yes
1010,NaN,930634438655,NaN,M.Valli,Girls Cycle,2023,Public,Yes
1011,NaN,408937849173,NaN,P.Gangadharan,Tab,2023,Public,Yes


In [ ]:
public2 = pd.read_excel("DISTRICT AND PUBLIC DATA BASE 31012024.xlsx",sheet_name="DO NOT TOUCH_2021 to 23 data")
public2

In [13]:
public1[public1["AADHAR No.1"]== "780149866273"][["NAME","BENEFICIARY ITEM","YEAR","Selection Status\nYes / No"]]

,NAME,BENEFICIARY ITEM,YEAR,Selection Status\nYes / No
1002,T.Vigneswar,Laptop,2023,Yes


In [12]:
public1.columns


Index(['AADHAR No', 'AADHAR No.1', 'Len', 'NAME', 'BENEFICIARY ITEM', 'YEAR',
       'Category', 'Selection Status\nYes / No'],
      dtype='object')